#### Training temporally aggregated ARIMA models for FCR estimation to compare its performance with our vehicle-specific metamodels at low-res scales
#### Ehsan Moradi, Ph.D. Candidate

In [33]:
# Load required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score


In [34]:
# General settings
VEHICLES = (
    "019 Hyundai Elantra GT 2019 (2.0L Auto)",
    "025 Chevrolet Captiva 2010 (2.4L Auto)",
    "027 Chevrolet Cruze 2011 (1.8L Manual)",
)
FEATURES = ["SPD_KH", "ACC_MS2", "ALT_M"]
DEPENDENT = "FCR_LH"
SETTINGS = {
    "INPUT_TYPE": "ENSEMBLE",
    "INPUT_INDEX": "06",
    "OUTPUT_TYPE": "ARIMA",
    "OUTPUT_INDEX": "AGGREGATE",
}
AGG_CFG = {"DATETIME": "last", "ALT_M": "mean", "SPD_KH": "mean",
           "FCR_LH": "mean", "ACC_MS2": "mean", "FCR_LH_PRED_METAMODEL": "mean"}


In [35]:
# Load sample data from Excel to a pandas dataframe
def load_from_Excel(vehicle, sheet, settings):
    directory = (
        "../../Academia/PhD/Field Experiments/Veepeak/"
        + vehicle
        + "/Processed/"
        + settings["INPUT_TYPE"]
        + "/"
    )
    input_file = vehicle + " - {0} - {1}.xlsx".format(
        settings["INPUT_TYPE"], settings["INPUT_INDEX"]
    )
    input_path = directory + input_file
    df = pd.read_excel(input_path, sheet_name=sheet, header=0)
    return df


In [36]:
# Save the predicted field back to Excel file
def save_to_excel(df, vehicle, scale, settings):
    directory = (
        "../../Academia/PhD/Field Experiments/Veepeak/"
        + vehicle
        + "/Processed/"
        + settings["OUTPUT_TYPE"]
        + "/"
    )
    output_file = vehicle + " - {0} - {1} - {2}-SEC.xlsx".format(
        settings["OUTPUT_TYPE"], settings["OUTPUT_INDEX"], scale
    )
    output_path = directory + output_file
    with pd.ExcelWriter(output_path, engine="openpyxl", mode="w") as writer:
        df.to_excel(writer, header=True, index=None)
    print("{} -> Data is saved to Excel successfully!".format(vehicle))
    return None


In [ ]:
# Training the ARIMA model for three temporal scales (1-sec, 5-sec, and 10-sec)
# and generating out-of-sample predictions
predictions, observations = {}, {}
for vehicle in VEHICLES:
    predictions[vehicle], observations[vehicle] = {}, {}
    df = load_from_Excel(vehicle, "Sheet1", SETTINGS)
    df = df[["DATETIME", "ALT_M", "SPD_KH", "FCR_LH",
             "ACC_MS2", "FCR_LH_PRED_METAMODEL"]]
    for scale in [1, 5, 10]:
        dfs = df.groupby(df.index // scale).agg(AGG_CFG)
        # Apply feature scaling
        scaler_features = StandardScaler().fit(dfs[FEATURES])
        scaler_dependent = StandardScaler().fit(dfs[[DEPENDENT]])
        dfs[FEATURES] = scaler_features.transform(dfs[FEATURES])
        dfs[[DEPENDENT]] = scaler_dependent.transform(dfs[[DEPENDENT]])
        # Train-Test splitting (70%-30%)
        split_point = int(.7 * len(dfs))
        train = dfs[:split_point].copy(deep=True)
        # Train the ARIMA model
        # The AR order is chosen as 6 (in accordance with our RNN modeling lag order)
        # As the variables could be considered stationary (they are bounded and trendless), "difference" is set to 0.
        # Moving-average order of 3 is applied.
        model_l6 = ARIMA(train[DEPENDENT],
                         exog=train[FEATURES], order=(6, 0, 3))
        fit_l6 = model_l6.fit(method_kwargs={"warn_convergence": False})
        # Out-of-sample prediction
        predictions[vehicle][scale] = fit_l6.predict(
            start=len(train), end=len(dfs) - 1, exog=dfs[FEATURES][split_point:]).values
        # Apply inverse scaling
        dfs[FEATURES] = scaler_features.inverse_transform(dfs[FEATURES])
        predictions[vehicle][scale] = scaler_dependent.inverse_transform(
            predictions[vehicle][scale])
        dfs[[DEPENDENT]] = scaler_dependent.inverse_transform(
            dfs[[DEPENDENT]])
        observations[vehicle][scale] = dfs[DEPENDENT][split_point:]
        dfs.loc[split_point:, "FCR_LH_PRED_ARIMA_{0}_SEC".format(
            scale)] = predictions[vehicle][scale]
        save_to_excel(dfs, vehicle, scale, SETTINGS)


In [ ]:
# Time-series plot of ARIMA predictions vs. true observations for a selected time-window
for vehicle in VEHICLES:
    for scale in [1, 5, 10]:
        fig, ax = plt.subplots(figsize=(12, 4))
        sns.lineplot(x=range(750), y=predictions[vehicle][scale][0:750], color="blue")
        sns.lineplot(x=range(750), y=observations[vehicle][scale][0:750], color="red")
        plt.legend(labels=["Predictions (AR Order = 6)", "True Observations"])
        plt.show()


In [ ]:
# Scatter plot to compare ARIMA predictions and true observations
for vehicle in VEHICLES:
    for scale in [1, 5, 10]:
        fig, ax = plt.subplots(figsize=(4, 4))
        sns.scatterplot(x=observations[vehicle][scale], y=predictions[vehicle][scale])
        upper_bound = np.max([np.max(observations[vehicle][scale]),
                            np.max(predictions[vehicle][scale])])
        plt.xlim(0, upper_bound)
        plt.ylim(0, upper_bound)
        plt.xlabel("True Observations")
        plt.ylabel("ARIMA Predictions (AR Order = 6)")
        plt.show()


In [39]:
# Calculate R-squared score of scaled ARIMA models
for vehicle in VEHICLES:
    for scale in [1, 5, 10]:
        print("{0}, {1}: {2}".format(vehicle, scale, r2_score(
            observations[vehicle][scale], predictions[vehicle][scale])))


019 Hyundai Elantra GT 2019 (2.0L Auto), 1: 0.5256432994136848
019 Hyundai Elantra GT 2019 (2.0L Auto), 5: 0.6599507495708109
019 Hyundai Elantra GT 2019 (2.0L Auto), 10: 0.7143308152151611
025 Chevrolet Captiva 2010 (2.4L Auto), 1: 0.10790025568393047
025 Chevrolet Captiva 2010 (2.4L Auto), 5: 0.24965064042552265
025 Chevrolet Captiva 2010 (2.4L Auto), 10: 0.22913369857489152
027 Chevrolet Cruze 2011 (1.8L Manual), 1: 0.5548563597332341
027 Chevrolet Cruze 2011 (1.8L Manual), 5: 0.6666841247201505
027 Chevrolet Cruze 2011 (1.8L Manual), 10: 0.703432450942992


In [40]:
# Load sample data from Excel to a pandas dataframe (for scaled predictions)
def load_from_Excel_scaled(vehicle, scale, sheet):
    directory = (
        "../../Academia/PhD/Field Experiments/Veepeak/"
        + vehicle
        + "/Processed/ARIMA/"
    )
    input_file = vehicle + " - ARIMA - AGGREGATE - {0}-SEC.xlsx".format(scale)
    input_path = directory + input_file
    df = pd.read_excel(input_path, sheet_name=sheet, header=0)
    return df


In [42]:
# Calculate R-square score for scaled metamodel predictions
for vehicle in VEHICLES:
    for scale in [1, 5, 10]:
        df = load_from_Excel_scaled(vehicle, scale, "Sheet1")
        print("{0}, {1}: {2}".format(vehicle, scale, r2_score(
            df["FCR_LH"], df["FCR_LH_PRED_METAMODEL"])))


019 Hyundai Elantra GT 2019 (2.0L Auto), 1: 0.7179144438996059
019 Hyundai Elantra GT 2019 (2.0L Auto), 5: 0.827947997358645
019 Hyundai Elantra GT 2019 (2.0L Auto), 10: 0.8397226533437434
025 Chevrolet Captiva 2010 (2.4L Auto), 1: 0.8648514862384631
025 Chevrolet Captiva 2010 (2.4L Auto), 5: 0.9226837940291782
025 Chevrolet Captiva 2010 (2.4L Auto), 10: 0.9404964902260587
027 Chevrolet Cruze 2011 (1.8L Manual), 1: 0.7647982784034113
027 Chevrolet Cruze 2011 (1.8L Manual), 5: 0.8331106614564179
027 Chevrolet Cruze 2011 (1.8L Manual), 10: 0.8440208423245168
